# Coursera Captsone
This will mainly be used for the coursera capstone project. with longLat data.

In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from pandas.io.json import json_normalize


In [3]:
#Scrape website 

#set URL to Search, get HTML of page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

#pass html into beautifulSoup and create beautifulsoup object named soup
soup = BeautifulSoup(html, 'lxml')

#check if the text looks good by print(text)
text = soup.get_text()

#find all within page you have scraped
table = soup.find("table", class_="wikitable sortable")


#print(table.prettify)

In [4]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df = pd.read_html(str(table))[0]
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [5]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df2 = df[df.Borough != "Not assigned"]

df2.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
#Replace a neighbourhood that is "not assigned with the value of the borough column
df2.Neighbourhood[df2.Neighbourhood == 'Not assigned'] = df.Borough
df2.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
#Group based on Postcode and borough
df3 = df2.groupby(["Postcode", "Borough"])['Neighbourhood'].apply(list)
df4 = df3.to_frame().reset_index()

#Convert list to string to remove brackets
df4['Neighbourhood'] = [', '.join(map(str, l)) for l in df4['Neighbourhood']]
df4.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df4.shape

(103, 3)

## Next step - Importing Geocode data

In [9]:
#Wasnt happy with the integrity of data using the Geocode import- using CSV instead
pc_data = pd.read_csv('Geospatial_Coordinates.csv')
pc_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
pd_full = pd.merge(df4, pc_data, left_on='Postcode', right_on='Postal Code')
pdfinal = pd_full.drop(['Postal Code'], axis=1)
pdfinal.head(10)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Clustering

In [27]:
#Import Additional Libraries
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.4.1               |           py36_0         537 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.5 MB

The fol

In [57]:
#one hot encode data set
# one hot encoding
can_onehot = pd.get_dummies(pdfinal[['Borough']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
can_onehot['Neighbourhood'] = pdfinal['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [can_onehot.columns[-1]] + list(can_onehot.columns[:-1])
can_onehot = can_onehot[fixed_columns]

can_onehot.head()

,Neighbourhood,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,1,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,1,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,1,0,0
3,Woburn,0,0,0,0,0,0,0,0,1,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,1,0,0


In [58]:
can_grouped = can_onehot.groupby('Neighbourhood').mean().reset_index()
can_grouped.head()



,Neighbourhood,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,"Adelaide, King, Richmond",0,1,0,0,0,0,0,0,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,1,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,0,0,0,0,0,1,0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,0,0,0,1,0,0,0,0,0,0
4,"Alderwood, Long Branch",0,0,0,0,1,0,0,0,0,0,0


In [78]:
# set number of clusters
numclusters = 10

cluster_data = can_grouped.drop(['Neighbourhood'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=numclusters, random_state=0).fit(cluster_data)

# check cluster labels generated for each row in the dataframe
km = kmeans.labels_
cluster_data.head()

,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0


In [80]:
# add clustering labels
#pdfinal = pdfinal.drop(['Cluster Labels'], axis=1)
pdfinal.insert(0, 'Cluster Labels', kmeans.labels_)
can_merged = pdfinal
can_merged.head() # check the last columns!

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,2,M1G,Scarborough,Woburn,43.770992,-79.216917
4,2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [31]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done

## Package Plan ##

  environment location: /home/nbuser/anaconda3_501

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.18.1-py_0

The following packages wi

In [81]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(numclusters)
ys = [i + x + (i*x)**2 for i in range(numclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(can_merged['Latitude'], can_merged['Longitude'], can_merged['Neighbourhood'], can_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters